In [107]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

In [91]:
print("my 1st OOF...never give up")
SEED = 7    # CR7 always..
path = 'Data/'

application_train = pd.read_csv(path + "application_train.csv")
application_test = pd.read_csv(path + "application_test.csv")
POS_CASH = pd.read_csv(path + 'POS_CASH_balance.csv')
credit_card = pd.read_csv(path + 'credit_card_balance.csv')
bureau = pd.read_csv(path + 'bureau.csv')
bureau_balance = pd.read_csv(path + 'bureau_balance.csv')
previous_app = pd.read_csv(path + 'previous_application.csv')
installments = pd.read_csv(path + 'installments_payments.csv')
subm = pd.read_csv(path+ "sample_submission.csv")

my 1st OOF...never give up


In [92]:
#POS_CASH = pd.read_csv(path+ 'POS_CASH_balance.csv')
print("Converting...")
le = LabelEncoder()
POS_CASH['NAME_CONTRACT_STATUS'] = le.fit_transform(POS_CASH['NAME_CONTRACT_STATUS'].astype(str))
nunique_status = POS_CASH[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').nunique()
count_status = POS_CASH.groupby('SK_ID_CURR').size()
#print(count_status)
#print(nunique_status)
POS_CASH['NUNIQUE_STATUS'] = nunique_status['NAME_CONTRACT_STATUS']
POS_CASH['pc_COUNT_STATUS'] = count_status
POS_CASH.drop(['SK_ID_PREV', 'NAME_CONTRACT_STATUS'], axis=1, inplace=True)
#print(POS_CASH[POS_CASH['NUNIQUE_STATUS'].notna()])

Converting...


In [93]:
#credit_card = pd.read_csv(path+ 'credit_card_balance.csv')

credit_card['NAME_CONTRACT_STATUS'] = le.fit_transform(credit_card['NAME_CONTRACT_STATUS'].astype(str))
nunique_status = credit_card[['SK_ID_CURR', 'NAME_CONTRACT_STATUS']].groupby('SK_ID_CURR').nunique()
count_status = credit_card.groupby('SK_ID_CURR').size()
credit_card['NUNIQUE_STATUS'] = nunique_status['NAME_CONTRACT_STATUS']
credit_card['cc_COUNT_STATUS'] = count_status
credit_card.drop(['SK_ID_PREV', 'NAME_CONTRACT_STATUS'], axis=1, inplace=True)

In [94]:
bureau_balance = pd.read_csv(path + 'bureau_balance.csv')
bureau_balance['STATUS'] = le.fit_transform(bureau_balance['STATUS'].astype(str))
unique_status =bureau_balance[['SK_ID_BUREAU','STATUS']].groupby('SK_ID_BUREAU').nunique()
count_status = bureau_balance.groupby('SK_ID_BUREAU').size()
bureau_balance['NUNIQUE_STATUS'] = unique_status['STATUS']
bureau_balance['bb_COUNT_STATUS'] = count_status
bureau_balance.drop(['STATUS'], axis=1, inplace=True)

bureau = pd.read_csv(path+ 'bureau.csv')
bureau = bureau.merge(bureau_balance.groupby('SK_ID_BUREAU').mean().reset_index(), 
                                                      how = 'left', on= 'SK_ID_BUREAU')

In [95]:
count_status = bureau.groupby('SK_ID_CURR').size()
bureau['B_COUNT_STATUS'] = count_status
bureau_cat_features = [f for f in bureau.columns if bureau[f].dtype == 'object']
for f in bureau_cat_features:
    bureau[f] = le.fit_transform(bureau[f].astype(str))
    nunique = bureau[['SK_ID_CURR', f]].groupby('SK_ID_CURR').nunique()
    bureau['NUNIQUE_'+f] = nunique[f]
    bureau.drop([f], axis=1, inplace=True)
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)

In [96]:
count_status = previous_app.groupby('SK_ID_CURR').size()
previous_app['pa_COUNT_STATUS'] = count_status

previous_app_cat_features = [f for f in previous_app.columns if previous_app[f].dtype == 'object']
for f in previous_app_cat_features:
    previous_app[f] = le.fit_transform(previous_app[f].astype(str))
    nunique = previous_app[['SK_ID_CURR', f]].groupby('SK_ID_CURR').nunique()
    previous_app['NUNIQUE_'+f] = nunique[f]
    previous_app.drop([f], axis=1, inplace=True)
previous_app.drop(['SK_ID_PREV'], axis=1, inplace=True)

In [97]:
count_status = installments.groupby('SK_ID_CURR').size()
installments['ip_COUNT_STATUS'] = count_status

installments.drop(['SK_ID_PREV'], axis=1 , inplace = True)

In [98]:
print("Merging...")
data_train = application_train.merge(POS_CASH.groupby('SK_ID_CURR').mean().reset_index(),
                                                             how='left', on='SK_ID_CURR')
data_test = application_test.merge(POS_CASH.groupby('SK_ID_CURR').mean().reset_index(),
                                                           how='left', on='SK_ID_CURR')

data_train = data_train.merge(credit_card.groupby('SK_ID_CURR').mean().reset_index(),
                                                         how='left', on='SK_ID_CURR')
data_test = data_test.merge(credit_card.groupby('SK_ID_CURR').mean().reset_index(),
                                                       how='left', on='SK_ID_CURR')
                                                       
data_train = data_train.merge(bureau.groupby('SK_ID_CURR').mean().reset_index(),
                                                    how='left', on='SK_ID_CURR')
data_test = data_test.merge(bureau.groupby('SK_ID_CURR').mean().reset_index(),
                                                  how='left', on='SK_ID_CURR')
                                                  
data_train = data_train.merge(previous_app.groupby('SK_ID_CURR').mean().reset_index(),
                                                          how='left', on='SK_ID_CURR')
data_test = data_test.merge(previous_app.groupby('SK_ID_CURR').mean().reset_index(),
                                                        how='left', on='SK_ID_CURR')

data_train = data_train.merge(installments.groupby('SK_ID_CURR').mean().reset_index(),
                                                         how = 'left', on='SK_ID_CURR')
data_test =  data_test.merge(installments.groupby('SK_ID_CURR').mean().reset_index(),
                                                        how = 'left', on='SK_ID_CURR')


Merging...


In [99]:
target_train = data_train['TARGET']
data_train.drop(['SK_ID_CURR', 'TARGET'], axis=1, inplace=True)
data_test.drop(['SK_ID_CURR'], axis=1, inplace=True)

In [101]:
cat_features = [f for f in data_train.columns if data_train[f].dtype == 'object']
def column_index(df, query_cols):
    cols = df.columns.values  ## col name을 array형태로 return 합니다.
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, query_cols, sorter=sidx)]
cat_features_inds = column_index(data_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_inds)

Cat features are: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
[ 0  1  2  3  9 10 11 12 13 26 30 38 84 85 87 88]


In [102]:
for col in cat_features:
    data_train[col] = le.fit_transform(data_train[col].astype(str))
    data_test[col] = le.fit_transform(data_test[col].astype(str))
    
data_train.fillna(-1, inplace=True)
data_test.fillna(-1, inplace=True)
cols = data_train.columns

ntrain = data_train.shape[0]
ntest = data_test.shape[0]

print(data_train.shape)

(307511, 211)


In [103]:
kf = KFold(data_train.shape[0], n_folds=5, shuffle=True, random_state=7)
NFOLDS = 5
x_train = np.array(data_train)
x_test = np.array(data_test)
y_train = target_train.ravel()

In [108]:
# from https://www.kaggle.com/mmueller/stacking-starter?scriptVersionId=390867/code
class SklearnWrapper(object):
    def __init__(self, clf, seed=7, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        print("Training..")
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        print("Predicting..")
        return self.clf.predict_proba(x)


class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        print("Training..")
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        print("Predicting..")
        return self.gbdt.predict(xgb.DMatrix(x))

class lightgbmWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        self.seed = seed
        
    def train(self, x_tr, y_tr):
        x_train, x_val, y_train, y_val = train_test_split(x_tr, y_tr, random_state=self.seed)
        lgb_train = lgb.Dataset(data=x_train, label=y_train)
        lgb_eval = lgb.Dataset(data=x_val, label=y_val)
        print("Training..")
        self.lbgm = lgb.train(self.param, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=100, verbose_eval=10)

    def predict(self, x):
        print("Predicting..")
        return self.lbgm.predict(x)
    

def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)[:,1]  # or [:,0]
        oof_test_skf[i, :] = clf.predict(x_test)[:,1]  # or [:,0]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)
    
def get_oof_xgb(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)  # or [:,0]
        oof_test_skf[i, :] = clf.predict(x_test)  # or [:,0]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


In [109]:
et_params = {
    'n_jobs': 16,
    'n_estimators': 100,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 100,
    'max_features': 0.2,
    'max_depth': 8,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'reg:linear',
    'max_depth': 7,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'eval_metric': 'mae',
    'nrounds': 350
}

cb_params = {
    'iterations':1000,
    'learning_rate':0.1,
    'depth':6,
    'l2_leaf_reg':40,
    'bootstrap_type':'Bernoulli',
    'subsample':0.7,
    'scale_pos_weight':5,
    'eval_metric':'AUC',
    'metric_period':50,
    'od_type':'Iter',
    'od_wait':45,
    'allow_writing_files':False    
}

lgbm_params = {'task': 'train', 
          'boosting_type': 'gbdt', 
          'objective': 'binary', 
          'metric': 'auc', 
          'learning_rate': 0.01, 
          'num_leaves': 32, 
          'num_iteration': 500,
          'verbose': 0 }

In [110]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = SklearnWrapper(clf=CatBoostClassifier, seed=SEED, params=cb_params)
lg = lightgbmWrapper(seed=SEED, params=lgbm_params)


print("xg..")
xg_oof_train, xg_oof_test = get_oof_xgb(xg)
print("et..")
et_oof_train, et_oof_test = get_oof(et)
print("rf..")
rf_oof_train, rf_oof_test = get_oof(rf)
print("cb..")
cb_oof_train, cb_oof_test = get_oof(cb)
print('lg..')
lg_oof_train, lg_oof_test = get_oof_xgb(lg)

xg..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
et..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
rf..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
Training..
Predicting..
Predicting..
cb..
Training..
0:	learn: 0.7077527	total: 372ms	remaining: 6m 11s
50:	learn: 0.7693927	total: 14.7s	remaining: 4m 33s
100:	learn: 0.7830352	total: 29.5s	remaining: 4m 22s
150:	learn: 0.7914561	total: 43.7s	remaining: 4m 5s
200:	learn: 0.7989706	total: 57.7s	remaining: 3m 49s
250:	learn: 0.8058141	total: 1m 12s	remaining: 3m 37s
300:	learn: 0.8124329	total: 1m 26s	remaining: 3m 21s
350:	learn: 0.8181271	total: 1m 40s	remaining

Training..
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.713617
[20]	valid_0's auc: 0.717585
[30]	valid_0's auc: 0.720755
[40]	valid_0's auc: 0.722179
[50]	valid_0's auc: 0.723439
[60]	valid_0's auc: 0.724521
[70]	valid_0's auc: 0.725384
[80]	valid_0's auc: 0.726064
[90]	valid_0's auc: 0.726868
[100]	valid_0's auc: 0.727556
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.727556
Predicting..
Predicting..
Training..
Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.710544
[20]	valid_0's auc: 0.712372
[30]	valid_0's auc: 0.71698
[40]	valid_0's auc: 0.719158
[50]	valid_0's auc: 0.72069
[60]	valid_0's auc: 0.721883
[70]	valid_0's auc: 0.723047
[80]	valid_0's auc: 0.724394
[90]	valid_0's auc: 0.725087
[100]	valid_0's auc: 0.725839
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.725839
Predicting..
Predicting..
Training..
Training until validation scores don't improve for 10

In [111]:
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train,lg_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test,lg_oof_test), axis=1)
print(x_train[0:5,:])

np.save('x_train', x_train)
np.save('x_test', x_test)
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test)
print(dtrain)

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.8,
    'silent': 1,
    'subsample': 0.6,
    'learning_rate': 0.01,
    'objective': 'reg:linear',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'eval_metric': 'mae',
}

print("xgb cv..")
res = xgb.cv(xgb_params, dtrain, num_boost_round=500, nfold=4, seed=SEED, stratified=False,
             early_stopping_rounds=25, verbose_eval=10, show_stdv=True)
best_nrounds = res.shape[0] - 1

"""
print("\nCatBoost...")                                     
cb_model = CatBoostClassifier(iterations=1000,
                              learning_rate=0.1,
                              depth=6,
                              l2_leaf_reg=40,
                              bootstrap_type='Bernoulli',
                              subsample=0.7,
                              scale_pos_weight=5,
                              eval_metric='AUC',
                              metric_period=50,
                              od_type='Iter',
                              od_wait=45,
                              random_seed=17,
                              allow_writing_files=False)

cb_model.fit(X_train, y_train,
             eval_set=(X_valid, y_valid),
             cat_features=cat_features_inds,
             use_best_model=True,
             verbose=True)
             
print('AUC:', roc_auc_score(y_valid, cb_model.predict_proba(X_valid)[:,1]))
y_preds = cb_model.predict_proba(data_test)[:,1]
subm['TARGET'] = y_preds
subm.to_csv('submission.csv', index=False)
"""
print("meta xgb train..")
gbdt = xgb.train(xgb_params, dtrain, best_nrounds)
fi = gbdt.predict(dtest)
fi = np.array(fi)
np.save('fi', fi)

subm['TARGET'] = fi
subm.to_csv('stack3.csv', index=False)

[[0.35185713 0.23579652 0.19996994 0.75440015 0.34274418]
 [0.00668609 0.04861397 0.05323199 0.04562115 0.05088931]
 [0.01214725 0.04719649 0.03957637 0.11489188 0.211     ]
 [0.0030427  0.07010137 0.06923173 0.14372477 0.22247804]
 [0.06733778 0.12792115 0.11475297 0.2442684  0.26789903]]
xgb cv..
[0]	train-mae:0.496332+1e-06	test-mae:0.496334+4.58258e-06
[10]	train-mae:0.461597+2.29674e-05	test-mae:0.461612+2.64811e-05
[20]	train-mae:0.430177+4.30145e-05	test-mae:0.430206+3.84383e-05
[30]	train-mae:0.401768+5.19345e-05	test-mae:0.401809+6.95463e-05
[40]	train-mae:0.376059+5.82173e-05	test-mae:0.376118+9.10971e-05
[50]	train-mae:0.352809+6.71919e-05	test-mae:0.352884+0.000119744
[60]	train-mae:0.331778+7.25547e-05	test-mae:0.331867+0.000140422
[70]	train-mae:0.312746+7.91912e-05	test-mae:0.312847+0.000154476
[80]	train-mae:0.295532+8.36103e-05	test-mae:0.295648+0.000170459
[90]	train-mae:0.279954+9.33234e-05	test-mae:0.280089+0.000183862
[100]	train-mae:0.265878+0.000101743	test-mae:0